<a href="https://colab.research.google.com/github/cbonnin88/VifStream/blob/main/User_Health_360.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit
!pip install -q -g localtunnel


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -g


In [2]:
%%writefile vifstream_ops_tool.py
import streamlit as st
import pandas as pd
from google.cloud import bigquery
import plotly.express as px


# 1. Page Config & Branding
st.set_page_config(page_title='VifStream Ops',page_icon="📺",layout='wide')
@st.cache_data(ttl=600) # Cache for 10 minutes
def get_user_data(uid):
    client = bigquery.Client(project="poetic-maxim-486609-e5")
    query = f"""
    SELECT * FROM `poetic-maxim-486609-e5.vifstream_dbt.fct_user_retention`
    WHERE CAST(user_id AS STRING) = "{uid}"
    """
    return client.query(query).to_dataframe()

st.title('📺 VifStream User Health 360')
st.markdown('internal tool for Product Managers and Customer Success')

# 2. Sidebar Search
user_id = st.sidebar.text_input('Search User ID','101')

if user_id:
  # Initialize BigQuery
  client= bigquery.Client(project='poetic-maxim-486609-e5')

  # Query the table
  query = f"""
  SELECT * FROM `poetic-maxim-486609-e5.vifstream_dbt.fct_user_retention`
  WHERE CAST(user_id AS STRING) = "{user_id}"
  """
  try:
        df_fct = client.query(query).to_dataframe()

        if not df_fct.empty:
            col1, col2, col3 = st.columns(3)
            col1.metric("Account Tier", str(df_fct['plan_type'].iloc[0]).upper())
            col2.metric("Total Events", len(df_fct))

            # Health Logic
            health = "🟢 Healthy" if len(df_fct) > 5 else "🔴 High Churn Risk"
            col3.metric("User Status", health)

            st.subheader("Behavioral Timeline")
            fig = px.scatter(df_fct, x="event_at", y="event_name", color="event_name",
                             template="plotly_white")
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.warning(f"No data found for User ID: {user_id}")

  except Exception as e:
        st.error(f"BigQuery Error: {e}")

  if not df_fct.empty:
    # Create 3 Metric Columns
    col1,col2,col3 = st. columns(3)

    # Logic for User Health
    last_day = df_fct['days_since_signup'].max()
    event_count = len(df_fct)

    col1.metric('Account Tier', df_fct['plan_type'].iloc[0].upper())
    col2.metric('Total Events', event_count)

    # Color-coded Health Indicator
    health_status = '🟢 Healthy' if event_count > 5 else '🔴 High Churn Risk'
    col3.metric('User Health Status', health_status)

    # 3. Behavioral Timeline
    st.subheader('Event Timeline')
    fig = px.scatter(
        df_fct,
        x='event_at',
        y='event_name',
        color='event_name',
        height=300,
        template='plotly_white'
    )
    st.plotly_chart(fig,use_container_width=True)

    # 4. Actionable Advice for the Manager
    st.info(f'**Action Recommended:** {'No action neede. User is active.' if health_status == '🟢 Healthy' else 'Send retention email with 10% discount'}')

  else:
    st.error(f'User {user_id} not found in the BigQuery/Google Sheets dataset.')


Overwriting vifstream_ops_tool.py


In [3]:
from google.colab import auth
auth.authenticate_user()

import os
os.environ['GOOGLE_CLOUD_PROJECT'] = 'poetic-maxim-486609-e5'

In [5]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# 2. Run Streamlit in background
import os
os.system("streamlit run vifstream_ops_tool.py &")

# 3. Open the Tunnel
!cloudflared tunnel --url http://localhost:8501

--2026-02-11 09:53:24--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64.deb [following]
--2026-02-11 09:53:24--  https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/4fddf4d7-e02d-44dc-9e5a-ef9e28afdd54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-11T10%3A31%3A40Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&

In [4]:
!pkill streamlit
!pkill npx